In [68]:
import math
import itertools
def merge_consecutive_duplicates(lst):
    merged_list = []
    prev_element = None
    
    for element in lst:
        if element != prev_element:
            merged_list.append(element)
            prev_element = element
    
    return merged_list
def euclid_distance(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

def evenGaps(clusters, cluster_order, total_volume, total_spaces, distance_to_point, expanded_cluster=None):
    border_gaps = [0 for _ in range(total_spaces)]
    border_points = []
    cell_width = math.sqrt(1/total_spaces)
    gaps = []
    accumulative_gap = 0
    accumulative_filled = 0 
    spaces = {}
    if expanded_cluster:
        # remove expanded cluster from cluster_order
        parent_cluster_label = expanded_cluster['parent']
        sub_cluster_labels = expanded_cluster['sub_clusters']
        replaced_cluster_order = [parent_cluster_label if cluster_label in sub_cluster_labels else cluster_label for cluster_label in cluster_order]
        merged_cluster_order = merge_consecutive_duplicates(replaced_cluster_order)
        clusters[parent_cluster_label] = list(itertools.chain(*[clusters[sub_cluster_label] for sub_cluster_label in sub_cluster_labels]))
        # calculate spacing for merged cluster order
        for cluster_label in merged_cluster_order:
            nodes = clusters[cluster_label]
            cluster_volume = len(nodes)
            ratio = cluster_volume / total_volume
            cluster_space = total_spaces * ratio
            spaces[cluster_label] = cluster_space
        # re-distribute the space of merged cluster evenly to sub clusters
        parent_cluster_volume = len(clusters[parent_cluster_label])
        parent_clusters_space = spaces[parent_cluster_label]
        for sub_cluster_label in sub_cluster_labels:
            nodes = clusters[sub_cluster_label]
            sub_cluster_volume = len(nodes)
            ratio = sub_cluster_volume / parent_cluster_volume
            sub_cluster_space = parent_clusters_space * ratio
            spaces[sub_cluster_label] = sub_cluster_space
        # remove parent cluster spaces from spaces
        del spaces[parent_cluster_label]
        del clusters[parent_cluster_label]
        # calculate gaps
        for cluster_label in cluster_order:
            cluster_space = spaces[cluster_label]
            cluster_volume = len(clusters[cluster_label])
            padding = (cluster_space - cluster_volume)/2
            gaps.append(math.floor(accumulative_gap + padding))
            accumulative_gap += 2*padding
        return gaps
    else:
        # calculate gaps
        for cluster_label in cluster_order:
            nodes = clusters[cluster_label]
            cluster_volume = len(nodes)
            ratio = cluster_volume / total_volume
            cluster_space = total_spaces * ratio
            padding = (cluster_space - cluster_volume) / 2
            occupied = [math.floor(accumulative_gap + padding), math.floor(accumulative_gap + padding + cluster_volume)]
            # check if the cluster is on the border
            padding_movement = 0
            min_distance = 100
            print("cluster size: ", occupied[1] - occupied[0])
            for previous_distance, cluster_point_distance in enumerate(range(occupied[0], occupied[1])):
                cluster_point = distance_to_point[cluster_point_distance]
                for border_point in border_points:
                    d = euclid_distance(border_point, cluster_point)
                    if d < min_distance:
                        min_distance = d
                    if d < cell_width/2:
                        padding_movement = previous_distance
            print("padding movement: ", padding_movement, min_distance, cell_width/2, total_spaces)
            padding += padding_movement
                # if border_gaps[cluster_point] == 1:
                #     padding += previous_distance
            # recalculate occupied
            occupied = [math.floor(accumulative_gap + padding), math.floor(accumulative_gap + padding + cluster_volume)]
            print(len(gaps), math.floor(accumulative_gap + padding), occupied)
            gaps.append(math.floor(accumulative_gap + padding))
            border_points += find_border_points(occupied[0], occupied[1], distance_to_point)
            # print("border points:", len(border_points))
            # border_distances = find_border_points(occupied[0], occupied[1], distance_to_point, point_to_distance)
            # for distance in border_distances:
            #     border_gaps[distance] = 1
            accumulative_gap += 2*padding
            accumulative_filled += cluster_volume
            print("============================")
        return gaps

def find_border_points(start, end, distance_to_point):
    print("finding border points", (start, end))
    # mapping = point_to_distance
    # curve_type = point_to_distance['curve_type']
    curve_type = "gosper"
    total_points = len(distance_to_point)
    if curve_type == 'gilbert':
        return [0, 0] 
    elif curve_type == 'gosper':
        offset_x = 1 / math.sqrt(total_points)
        offset_y = 1 / math.sqrt(total_points)
        print("offsets", offset_x, offset_y, total_points)
        kx = 0.6873313602390351
        ky = 9.935177303243627
        vertical_dx = math.sqrt((offset_y * offset_y) / (1 + ky * ky))
        vertical_dy = ky * vertical_dx
        horizontal_dx = math.sqrt((offset_x * offset_x) / (1 + kx * kx))
        horizontal_dy = kx * horizontal_dx
        border_offsets = [
            (-vertical_dx-horizontal_dx, -vertical_dy+horizontal_dy), 
            (-vertical_dx, -vertical_dy), 
            (-horizontal_dx, horizontal_dy), 
            (horizontal_dx, -horizontal_dy), 
            (vertical_dx, vertical_dy), 
            (vertical_dx+horizontal_dx, vertical_dy-horizontal_dy)
        ]
        border_points = []
        for distance in range(start, end):
            point = distance_to_point[distance]
            border_points += [(point[0]+border_offset[0], point[1]+border_offset[1]) for border_offset in border_offsets]
            # border_point_distances = [mapping[find_closest_point(border_point, distance_to_point)] for border_point in border_points]
        # return border_point_distances
        return border_points
    else:
        print("error!")

def find_closest_point(point, curve_points):
    min_distance = 100000
    closest_point = (0, 0)
    for curve_point in curve_points:
        distance = (point[0] - curve_point[0]) ** 2 + (point[1] - curve_point[1]) ** 2
        if distance == 0: return curve_point
        if distance < min_distance:
            closest_point = curve_point
            min_distance = distance
    return closest_point


In [59]:
import json
args = json.load(open('evenGaps_args.json'))
clusters = args['clusters']
cluster_order = args['cluster_order']
total_volume = args['total_volume']
total_spaces = args['total_spaces']
sfc_points = args['sfc_points']
sfc_points = [tuple(point) for point in sfc_points]
# save_point_to_distance = args['point_to_distance']
expanded_cluster = args['expanded_cluster']
# point_to_distance = {}
# for point_str, distance in save_point_to_distance.items():
#     point = tuple(map(float, point_str.split('-')))
#     point_to_distance[point] = distance

In [69]:
evenGaps(clusters, cluster_order, total_volume, total_spaces, sfc_points, expanded_cluster)

cluster size:  1893
padding movement:  0 100 0.003856665604626809 16808
0 1162 [1162, 3055]
finding border points (1162, 3055)
offsets 0.007713331209253617 0.007713331209253617 16808
border points: 11358
cluster size:  22
padding movement:  21 0.0010193506806001403 0.003856665604626809 16808
1 2360 [2360, 2382]
finding border points (2360, 2382)
offsets 0.007713331209253617 0.007713331209253617 16808
border points: 11490
cluster size:  1087
padding movement:  1081 0.0010193506806000017 0.003856665604626809 16808
2 4143 [4143, 5230]
finding border points (4143, 5230)
offsets 0.007713331209253617 0.007713331209253617 16808
border points: 18012
cluster size:  3624
padding movement:  400 0.001019350680600467 0.003856665604626809 16808
3 8518 [8518, 12142]
finding border points (8518, 12142)
offsets 0.007713331209253617 0.007713331209253617 16808
border points: 39756
cluster size:  776
padding movement:  773 0.0010193506805999866 0.003856665604626809 16808
4 12394 [12394, 13170]
finding bor

[1162, 2360, 4143, 8518, 12394, 13777]